# Loading in Data

Loading the scrapped data to pandas

In [ ]:
import pandas as pd
import glob
import os

path = ""
all_files = glob.glob(os.path.join(path, "*.csv"))
activity_df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
activity_df['Created'] = pd.to_datetime(activity_df['Created'])
activity_df['Updated'] = pd.to_datetime(activity_df['Updated'])
pr_df = activity_df[["ID", "SrcRepo", "SrcBranch", "DestRepo", "DestBranch", "Title", "Description", "State", "Author", "Created", "Updated",
			"FileChanged", "Added", "Removed", "Total"]].copy().drop_duplicates()

Make sure the data is loaded properly and do column manipulation for further analysis

In [ ]:
activity_df['DateGroup'] = activity_df['Updated'].dt.strftime('%Y-%m')
activity_df.head()

In [ ]:
pr_df['DateGroup'] = pr_df['Updated'].dt.strftime('%Y-%m')
pr_df.head()

# PR Data Analysis

## Grouped by Repositories

In [ ]:
def pr_total_group_by(df,myList= [],*args): 
    c = ['Added','Removed','Total']
    return df.groupby(myList)[c].sum()

pr_total_group_by(pr_df, ['DateGroup', 'SrcRepo'])

## Grouped by Authors

In [ ]:
pr_total_group_by(pr_df, ['DateGroup', 'Author'])

# PR Activity Analysis

## Group by Repositories

In [ ]:
def activity_describe_group_by(df,myList= [],*args): 
    return df.groupby(myList)['Type'].count()

activity_describe_group_by(activity_df, ['DateGroup', 'SrcRepo', 'Type'])

## Group by Users

In [ ]:
activity_describe_group_by(activity_df, ['DateGroup', 'User', 'Type'])